I made too much simple LightGBM & XGboosting & NN Blend model.
https://www.kaggle.com/artgor helped me.

I'm beginner, so there may be many strange point.
Please give me a advise.

--------------------------------------------------------------

LightGBMとXGboostingとNNの混合によるシンプルな回帰モデルを構築しました。
https://www.kaggle.com/artgor　さんのkernelsを参考にさせていただきました。

日本語で書かれたkernelsがほとんどなかったため
少しでも初心者の助けになればとこちらのkernelsを作成します。

なお、私自身も初心者ですので理解のできていない点が多くあるかと思います。
ご指摘やアドバイスがあれば是非お願いします。

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Ridge, RidgeCV
import gc
from catboost import CatBoostRegressor
import seaborn as sns
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))


Loading Data.
"time_to_failure" has slight differences, so read by np.float64.

-------------------

データの読み込み。
"time_to_failure"はデータの値の差が非常に小さいので、np.float64で読み込む。

In [ ]:
X = pd.read_csv("../input/train.csv", nrows = 600000000, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

Making Features.

-----------------------

特徴量の作成。

In [ ]:
%time
import scipy

rows = 150_000
train = X
segments = int(np.floor(train.shape[0] / rows))

X_tr = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['ave', 'std', 'max', 'min',
                               'av_change_abs', 'av_change_rate', 'abs_max', 'abs_min',
                               'std_first_50000', 'std_last_50000', 'std_first_10000', 'std_last_10000',
                               'avg_first_50000', 'avg_last_50000', 'avg_first_10000', 'avg_last_10000',
                               'min_first_50000', 'min_last_50000', 'min_first_10000', 'min_last_10000',
                               'max_first_50000', 'max_last_50000', 'max_first_10000', 'max_last_10000',
                               'skew','skew_first_10000','skew_last_10000',
                                'skew_first_50000','skew_last_50000',
                                'kurt','kurt_first_10000','kurt_last_10000',
                                'kurt_first_50000','kurt_last_50000'])
y_tr = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['time_to_failure'])

total_mean = train['acoustic_data'].mean()
total_std = train['acoustic_data'].std()
total_max = train['acoustic_data'].max()
total_min = train['acoustic_data'].min()
total_sum = train['acoustic_data'].sum()
total_abs_max = np.abs(train['acoustic_data']).sum()

for segment in tqdm_notebook(range(segments)):
    seg = train.iloc[segment*rows:segment*rows+rows]
    x = seg['acoustic_data'].values
    y = seg['time_to_failure'].values[-1]
    
    y_tr.loc[segment, 'time_to_failure'] = y
    X_tr.loc[segment, 'ave'] = x.mean()
    X_tr.loc[segment, 'std'] = x.std()
    X_tr.loc[segment, 'max'] = x.max()
    X_tr.loc[segment, 'min'] = x.min()
    
    
    X_tr.loc[segment, 'av_change_abs'] = np.mean(np.diff(x))
    X_tr.loc[segment, 'av_change_rate'] = np.mean(np.nonzero((np.diff(x) / x[:-1]))[0])
    X_tr.loc[segment, 'abs_max'] = np.abs(x).max()
    X_tr.loc[segment, 'abs_min'] = np.abs(x).min()
    
    X_tr.loc[segment, 'std_first_50000'] = x[:50000].std()
    X_tr.loc[segment, 'std_last_50000'] = x[-50000:].std()
    X_tr.loc[segment, 'std_first_10000'] = x[:10000].std()
    X_tr.loc[segment, 'std_last_10000'] = x[-10000:].std()
    
    X_tr.loc[segment, 'avg_first_50000'] = x[:50000].mean()
    X_tr.loc[segment, 'avg_last_50000'] = x[-50000:].mean()
    X_tr.loc[segment, 'avg_first_10000'] = x[:10000].mean()
    X_tr.loc[segment, 'avg_last_10000'] = x[-10000:].mean()
    
    X_tr.loc[segment, 'min_first_50000'] = x[:50000].min()
    X_tr.loc[segment, 'min_last_50000'] = x[-50000:].min()
    X_tr.loc[segment, 'min_first_10000'] = x[:10000].min()
    X_tr.loc[segment, 'min_last_10000'] = x[-10000:].min()
    
    X_tr.loc[segment, 'max_first_50000'] = x[:50000].max()
    X_tr.loc[segment, 'max_last_50000'] = x[-50000:].max()
    X_tr.loc[segment, 'max_first_10000'] = x[:10000].max()
    X_tr.loc[segment, 'max_last_10000'] = x[-10000:].max()
    
    X_tr.loc[segment, 'skew'] = scipy.stats.skew(x)
    X_tr.loc[segment, 'kurt'] = scipy.stats.kurtosis(x)
        
    X_tr.loc[segment, 'skew_first_10000'] = scipy.stats.skew(x[:10000])
    X_tr.loc[segment, 'skew_last_10000'] = scipy.stats.skew(x[:-10000])
        
    X_tr.loc[segment, 'kurt_first_10000'] = scipy.stats.kurtosis(x[:10000])
    X_tr.loc[segment, 'kurt_last_10000'] = scipy.stats.kurtosis(x[:-10000])
    
    X_tr.loc[segment, 'skew_first_50000'] = scipy.stats.skew(x[:50000])
    X_tr.loc[segment, 'skew_last_50000'] = scipy.stats.skew(x[:-50000])
        
    X_tr.loc[segment, 'kurt_first_50000'] = scipy.stats.kurtosis(x[:50000])
    X_tr.loc[segment, 'kurt_last_50000'] = scipy.stats.kurtosis(x[:-50000])

Normalize Features.

------------------------

作成した特徴量の正規化。

In [ ]:
scaler = StandardScaler()
scaler.fit(X_tr)
X_train_scaled = pd.DataFrame(scaler.transform(X_tr), columns=X_tr.columns)

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
print(y_tr['time_to_failure'].describe())
sns.distplot(y_tr['time_to_failure'], bins=500)

In [ ]:
X_train_scaled.head()

In [ ]:
var = 'skew_first_10000'
data = pd.concat([y_tr['time_to_failure'], X_train_scaled], axis=1)
data.plot.scatter(x=var, y='time_to_failure', ylim=(0,20));

In [ ]:
corrmat = data.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

In [ ]:
k = 10 #number of variables for heatmap
cols = corrmat.nlargest(k, 'time_to_failure')['time_to_failure'].index
cm = np.corrcoef(data[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
sns.set()
cols = ['time_to_failure', 'min', 'min_last_50000', 'min_last_10000', 'min_first_50000', 'min_first_10000', 'av_change_rate']
sns.pairplot(data[cols], size = 2.5)
plt.show();

Making data to predict.

--------------------------------

訓練データの作成。

In [ ]:
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id')
X_test = pd.DataFrame(columns=X_tr.columns, dtype=np.float64, index=submission.index)
plt.figure(figsize=(22, 16))

for i, seg_id in enumerate(tqdm_notebook(X_test.index)):
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    
    x = seg['acoustic_data'].values
    X_test.loc[seg_id, 'ave'] = x.mean()
    X_test.loc[seg_id, 'std'] = x.std()
    X_test.loc[seg_id, 'max'] = x.max()
    X_test.loc[seg_id, 'min'] = x.min()
        
    X_test.loc[seg_id, 'av_change_abs'] = np.mean(np.diff(x))
    X_test.loc[seg_id, 'av_change_rate'] = np.mean(np.nonzero((np.diff(x) / x[:-1]))[0])
    X_test.loc[seg_id, 'abs_max'] = np.abs(x).max()
    X_test.loc[seg_id, 'abs_min'] = np.abs(x).min()
    
    X_test.loc[seg_id, 'std_first_50000'] = x[:50000].std()
    X_test.loc[seg_id, 'std_last_50000'] = x[-50000:].std()
    X_test.loc[seg_id, 'std_first_10000'] = x[:10000].std()
    X_test.loc[seg_id, 'std_last_10000'] = x[-10000:].std()
    
    X_test.loc[seg_id, 'avg_first_50000'] = x[:50000].mean()
    X_test.loc[seg_id, 'avg_last_50000'] = x[-50000:].mean()
    X_test.loc[seg_id, 'avg_first_10000'] = x[:10000].mean()
    X_test.loc[seg_id, 'avg_last_10000'] = x[-10000:].mean()
    
    X_test.loc[seg_id, 'min_first_50000'] = x[:50000].min()
    X_test.loc[seg_id, 'min_last_50000'] = x[-50000:].min()
    X_test.loc[seg_id, 'min_first_10000'] = x[:10000].min()
    X_test.loc[seg_id, 'min_last_10000'] = x[-10000:].min()
    
    X_test.loc[seg_id, 'max_first_50000'] = x[:50000].max()
    X_test.loc[seg_id, 'max_last_50000'] = x[-50000:].max()
    X_test.loc[seg_id, 'max_first_10000'] = x[:10000].max()
    X_test.loc[seg_id, 'max_last_10000'] = x[-10000:].max()
    
    X_test.loc[seg_id, 'skew'] = scipy.stats.skew(x)
    X_test.loc[seg_id, 'kurt'] = scipy.stats.kurtosis(x)
        
    X_test.loc[seg_id, 'skew_first_10000'] = scipy.stats.skew(x[:10000])
    X_test.loc[seg_id, 'skew_last_10000'] = scipy.stats.skew(x[:-10000])
        
    X_test.loc[seg_id, 'kurt_first_10000'] = scipy.stats.kurtosis(x[:10000])
    X_test.loc[seg_id, 'kurt_last_10000'] = scipy.stats.kurtosis(x[:-10000])
            
    X_test.loc[seg_id, 'skew_first_50000'] = scipy.stats.skew(x[:50000])
    X_test.loc[seg_id, 'skew_last_50000'] = scipy.stats.skew(x[:-50000])
        
    X_test.loc[seg_id, 'kurt_first_50000'] = scipy.stats.kurtosis(x[:50000])
    X_test.loc[seg_id, 'kurt_last_50000'] = scipy.stats.kurtosis(x[:-50000])
    
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

Trainig and Predict by LightGBM.

--------------------------------------

LightGBMによる学習および予測。

In [ ]:
model = lgb.LGBMRegressor(n_estimators=10000,learning_rate=0.0005, n_jobs=-1)
model.fit(X_train_scaled,y_tr)
y_pred_lgb = model.predict(X_test_scaled)

Trainig and Predict by XGboosting.

--------------------------------------
XGboostingによる学習および予測。

In [ ]:
model = xgb.XGBRegressor(n_estimators=10)
model.fit(X_train_scaled,y_tr)
y_pred_xgb = model.predict(X_test_scaled)

Training and Predict by NN.

-------------------------------

NNによる学習および予測。

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
import tensorflow as tf

inputs = Input(shape=(34,))
x = Dense(128, activation='relu')(inputs)
x = Dense(128, activation='relu')(x)
predictions = Dense(1)(x)
model_3 = Model(inputs=inputs, outputs=predictions)
model_3.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
                      loss='mse',
                    metrics=['accuracy'])
model_3.fit(X_train_scaled, y_tr, epochs=15, verbose=0)
y_pred_nn = model.predict(X_test_scaled).flatten()

In [ ]:
X_test_scaled

Blending.

---------------------------

混合。

In [ ]:
sample = pd.read_csv("../input/sample_submission.csv")
sample['time_to_failure'] = (y_pred_lgb+y_pred_xgb+y_pred_nn)/3
sample.to_csv('submission.csv',index=False)

Cross Validation.

---------------------------

交差検証。

In [ ]:
%time

from sklearn.model_selection import StratifiedKFold, ShuffleSplit, cross_val_score
from sklearn.metrics import log_loss, roc_auc_score
from tqdm import tqdm
from statistics import mean

n_estimators = [5,10]

for n_est in  n_estimators:
    print(n_est)
    mae = []
    cv = ShuffleSplit(n_splits=5,random_state=0)
    for train, valid in cv.split(X_train_scaled, y_tr):
        x_train = X_train_scaled.iloc[train]
        x_valid = X_train_scaled.iloc[valid]
        y_train = y_tr.iloc[train]
        y_valid = y_tr.iloc[valid]
        
        model_1 = lgb.LGBMRegressor(n_estimators=10000, learning_rate=0.0005,n_jobs=-1,random_state=0)
        model_1.fit(x_train, y_train)
        y_pred_lgb = model_1.predict(x_valid)
        
        model_2 = xgb.XGBRegressor(n_estimators=n_est, n_jobs=-1,random_state=0)
        model_2.fit(x_train, y_train)
        y_pred_xgb = model_2.predict(x_valid)
        
        inputs = Input(shape=(34,))
        x = Dense(128, activation='relu')(inputs)
        x = Dense(128, activation='relu')(x)
        predictions = Dense(1)(x)
        model_3 = Model(inputs=inputs, outputs=predictions)
        model_3.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
                      loss='mse',
                    metrics=['accuracy'])
        model_3.fit(x_train, y_train, epochs=15, verbose=0)
        y_pred_nn = model_3.predict(x_valid).flatten()
        
        y_pred = (y_pred_lgb+y_pred_xgb+y_pred_nn)/3
        mae.append(mean_absolute_error(y_valid, y_pred))
    print(mean(mae))